In [1]:
!nvidia-smi

Fri Nov 14 00:55:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.06              Driver Version: 580.65.06      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200 NVL                On  |   00000001:E1:00.0 Off |                    0 |
| N/A   24C    P0             67W /  600W |       0MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [6]:
!rclone copy "goolge:O365_vehicle13_subset_2.zip" "/workspace" \
    --progress \
    --drive-chunk-size 128M


2025/11/14 01:05:20 ERROR : Google drive root '1LQ7IWDx1Ko5VAV9iOWw-iEUbJVv3rPxd': error reading source root directory: directory not found
Transferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Checks:                 0 / 0, -, Listed 7
Elapsed time:         0.2s
2025/11/14 01:05:20 ERROR : Attempt 1/3 failed with 1 errors and: directory not found
Transferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Errors:                 1 (retrying may help)
Checks:                 0 / 0, -, Listed 7
Elapsed time:         0.2s
2025/11/14 01:05:21 ERROR : Google drive root '1LQ7IWDx1Ko5VAV9iOWw-iEUbJVv3rPxd': error reading source root directory: directory not found
Transferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Checks:                 0 / 0, -, Listed 14
Elapsed time:         0.4s
2025/11/14 01:05:21 ERROR : Attempt 2/3 failed with 1 errors and: directory not found
Transferred:   	          0 B / 0 B, -, 0 B/s, ETA -
Errors:                 1 (retrying may help)
Checks:                 0 / 0, -

In [10]:
# 1. Installs the 7-Zip tool (which is multi-threaded)
!apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+transitional.1).
0 upgraded, 0 newly installed, 0 to remove and 74 not upgraded.


In [12]:
# 2. Extract using 7z. This will use multiple CPU cores.
# (x = extract, -o = output directory, no space)
!7z x /workspace/O365_vehicle13_subset_2.zip -o /workspace/O365_vehicle13_subset_2


7-Zip 23.01 (x64) : Copyright (c) 1999-2023 Igor Pavlov : 2023-06-20
 64-bit locale=C.UTF-8 Threads:512 OPEN_MAX:1048576

Scanning the drive for archives:
1 file, 126208459151 bytes (118 GiB)

Extracting archive: /workspace/O365_vehicle13_subset_2.zip
--
Path = /workspace/O365_vehicle13_subset_2.zip
Type = zip
Physical Size = 126208459151
64-bit = +
Characteristics = Zip64

Everything is Ok

Folders: 196
Files: 203895
Size:       194211630267
Compressed: 126208459151


In [13]:
import os
from pathlib import Path

# --- 1. Define your paths and strings ---

# The base directory where your files are located
base_dir = Path("/workspace/O365_vehicle13_subset_2")

# The list of files to check and update
files_to_update = [
    "o365_vehicle13_subset.yaml",
    "train_images.txt",
    "val_images.txt"
]

# The string you want to find (your old Windows path)
string_to_find = "F:/ultralytics_datasets/O365_vehicle13_subset_2"

# The string you want to replace it with (your new Linux path)
string_to_replace = "/workspace/O365_vehicle13_subset_2"

print(f"--- Starting path update in {base_dir} ---")

# --- 2. Loop through each file and perform the replacement ---

for filename in files_to_update:
    file_path = base_dir / filename
    
    if not file_path.exists():
        print(f"[NOT FOUND] File: {file_path}")
        continue

    try:
        # Read the file content
        with open(file_path, 'r') as f:
            content = f.read()
            
        # Perform the replacement
        if string_to_find in content:
            new_content = content.replace(string_to_find, string_to_replace)
            
            # Write the new content back to the file
            with open(file_path, 'w') as f:
                f.write(new_content)
                
            print(f"[UPDATED] File: {file_path}")
        else:
            print(f"[NO CHANGE] File: {file_path} (string not found)")

    except Exception as e:
        print(f"[ERROR] Could not process {file_path}: {e}")

print("--- Update process finished. ---")

--- Starting path update in /workspace/O365_vehicle13_subset_2 ---
[UPDATED] File: /workspace/O365_vehicle13_subset_2/o365_vehicle13_subset.yaml
[UPDATED] File: /workspace/O365_vehicle13_subset_2/train_images.txt
[UPDATED] File: /workspace/O365_vehicle13_subset_2/val_images.txt
--- Update process finished. ---


In [ ]:
!pip -q install -U ultralytics pycocotools tqdm rich

from google.colab import drive
drive.mount('/content/drive')

# Point Ultralytics "datasets" and "runs" to Drive
from ultralytics import settings
DATASETS_DIR = "/content/drive/MyDrive/ultralytics_datasets"
RUNS_DIR     = "/content/drive/MyDrive/ultralytics_runs"
settings.update({"datasets_dir": DATASETS_DIR, "runs_dir": RUNS_DIR})
print("Ultralytics datasets_dir:", settings["datasets_dir"])
print("Ultralytics runs_dir    :", settings["runs_dir"])

In [ ]:
# High-performance tools for parallel download/extraction
!apt -yqq update
!apt -yqq install aria2 p7zip-full pigz parallel > /dev/null

In [ ]:
from pathlib import Path

# Where to place Objects365 on your Drive
DRIVE_O365 = Path("/content/drive/MyDrive/ultralytics_datasets/Objects365")
ZIPS_DIR   = DRIVE_O365/"zips"
TRAIN_ZIPS = ZIPS_DIR/"train"
VAL_ZIPS   = ZIPS_DIR/"val"
IM_TRAIN   = DRIVE_O365/"images/train"
IM_VAL     = DRIVE_O365/"images/val"

# Create folders
for p in [TRAIN_ZIPS, VAL_ZIPS, IM_TRAIN, IM_VAL]:
    p.mkdir(parents=True, exist_ok=True)

print("Dataset root:", DRIVE_O365)

# ---- Concurrency knobs ----
# aria2c: connections per server, splits per file, simultaneous files
A2C_X = 16   # -x
A2C_S = 16   # -s
A2C_J = 16   # -j

# Extraction parallelism (how many tar extractions to run at once)
EXTRACT_J = 6  # try 6-8 on Pro; tune for IO speed

print(f"aria2c: -x{A2C_X} -s{A2C_S} -j{A2C_J} | extraction parallel -j {EXTRACT_J}")


In [ ]:
# Base URL (exactly as in YAML; URL-encoded Chinese chars kept)
BASE = "https://dorc.ks3-cn-beijing.ksyun.com/data-set/2020Objects365%E6%95%B0%E6%8D%AE%E9%9B%86"

train_ann_url = f"{BASE}/train/zhiyuan_objv2_train.tar.gz"
train_patch_urls = [f"{BASE}/train/patch{i}.tar.gz" for i in range(0, 51)]  # 0..50 inclusive

val_ann_url = f"{BASE}/val/zhiyuan_objv2_val.json"
val_v1_urls = [f"{BASE}/val/images/v1/patch{i}.tar.gz" for i in range(0, 16)]  # 0..15
val_v2_urls = [f"{BASE}/val/images/v2/patch{i}.tar.gz" for i in range(16, 44)] # 16..43

len(train_patch_urls), len(val_v1_urls), len(val_v2_urls)


In [ ]:
# Write URL lists to text files
(TRAIN_ZIPS/"train_patches.txt").write_text("\n".join(train_patch_urls))
(VAL_ZIPS/"val_patches.txt").write_text("\n".join(val_v1_urls + val_v2_urls))

print("Downloading TRAIN annotations (tar.gz) ->", DRIVE_O365)
!aria2c -x{A2C_X} -s{A2C_S} -j{A2C_J} -c -k1M "{train_ann_url}" -d "{DRIVE_O365}" -o "zhiyuan_objv2_train.tar.gz"

print("Downloading TRAIN image patches ->", TRAIN_ZIPS)
!aria2c -x{A2C_X} -s{A2C_S} -j{A2C_J} -c -k1M -i "{TRAIN_ZIPS}/train_patches.txt" -d "{TRAIN_ZIPS}"

print("Downloading VAL annotations (json) ->", DRIVE_O365)
!aria2c -x{A2C_X} -s{A2C_S} -j{A2C_J} -c -k1M "{val_ann_url}" -d "{DRIVE_O365}" -o "zhiyuan_objv2_val.json"

print("Downloading VAL image patches ->", VAL_ZIPS)
!aria2c -x{A2C_X} -s{A2C_S} -j{A2C_J} -c -k1M -i "{VAL_ZIPS}/val_patches.txt" -d "{VAL_ZIPS}"


In [ ]:
# Same paths you used before
import os
from pathlib import Path

DRIVE_O365 = Path("/content/drive/MyDrive/ultralytics_datasets/Objects365")
TRAIN_ZIPS = DRIVE_O365 / "zips/train"
VAL_ZIPS   = DRIVE_O365 / "zips/val"
IM_TRAIN   = DRIVE_O365 / "images/train"
IM_VAL     = DRIVE_O365 / "images/val"

IM_TRAIN.mkdir(parents=True, exist_ok=True)
IM_VAL.mkdir(parents=True, exist_ok=True)

print("TRAIN_ZIPS:", TRAIN_ZIPS)
print("VAL_ZIPS  :", VAL_ZIPS)
print("IM_TRAIN  :", IM_TRAIN)
print("IM_VAL    :", IM_VAL)


In [ ]:
import concurrent.futures, subprocess, glob, os, re
from tqdm import tqdm
from pathlib import Path

# --- helpers for parsing and selection ---

def _tar_basename(p: str) -> str:
    """Return filename without .tar.gz/.tgz."""
    name = os.path.basename(p)
    if name.endswith(".tar.gz"):
        return name[:-7]
    if name.endswith(".tgz"):
        return name[:-4]
    # Fallback
    return os.path.splitext(name)[0]

def _extract_index_from_name(name: str):
    """Use the LAST run of digits in the basename as the patch index (e.g., 'patch10' -> 10)."""
    m = re.search(r'(\d+)(?!.*\d)', _tar_basename(name))
    return int(m.group(1)) if m else None

def _select_from_index_lexicographic(files, start_index: int):
    """
    Keep lexicographic order (by filename) and return the slice starting from
    the first file whose parsed index == start_index. If not found, return all
    files whose parsed index >= start_index (still in lexicographic order).
    """
    files_sorted = sorted(files, key=lambda x: os.path.basename(x))
    pivot = None
    for i, f in enumerate(files_sorted):
        idx = _extract_index_from_name(f)
        if idx == start_index:
            pivot = i
            break
    if pivot is not None:
        selected = files_sorted[pivot:]
    else:
        selected = [f for f in files_sorted if (_extract_index_from_name(f) or -1) >= start_index]
    return selected

# --- your original extraction helpers ---

def extract_with_tar_skip(tar_path: str, dest_dir: str):
    os.makedirs(dest_dir, exist_ok=True)
    cmd = ["tar", "--skip-old-files", "-xzf", tar_path, "-C", dest_dir]
    # If GNU tar doesn't support --skip-old-files on your system, fall back:
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError:
        subprocess.run(["tar", "--keep-old-files", "-xzf", tar_path, "-C", dest_dir],
                       check=False, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

def extract_batch(pattern: str, dest_dir: str, workers: int = 1, desc: str = "", start_index: int = None):
    all_files = sorted(glob.glob(pattern), key=lambda x: os.path.basename(x))
    if not all_files:
        print(f"⚠️ No archives matched: {pattern}")
        return

    if start_index is not None:
        files = _select_from_index_lexicographic(all_files, start_index)
        if not files:
            print(f"⚠️ No files at or beyond start index {start_index} under pattern: {pattern}")
            return
        # Diagnostics (optional)
        print(f"Selected {len(files)} / {len(all_files)} files starting from index {start_index}")
        print("First few to extract:", ", ".join(os.path.basename(f) for f in files[:5]))
    else:
        files = all_files

    pbar = tqdm(total=len(files), desc=desc or f"Extract → {dest_dir}", unit="tar")
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as ex:
        futures = [ex.submit(extract_with_tar_skip, f, dest_dir) for f in files]
        for _ in concurrent.futures.as_completed(futures):
            pbar.update(1)
    pbar.close()

# ----------------- USAGE -----------------
# Assumes TRAIN_ZIPS, VAL_ZIPS, IM_TRAIN, IM_VAL are defined elsewhere
# Start TRAIN from '29' onward in *lexicographic* order, process all VAL
# extract_batch(str(Path(TRAIN_ZIPS) / "*.tar.gz"), str(IM_TRAIN),
#               workers=4, desc="TRAIN extract (29+; idempotent)", start_index=43)

extract_batch(str(Path(VAL_ZIPS) / "*.tar.gz"),   str(IM_VAL),
              workers=4, desc="VAL extract (all; idempotent)")


In [ ]:
!find /content/drive/MyDrive/ultralytics_datasets/Objects365/images/train/patch43 -maxdepth 1 -type f | wc -l
!du -sh /content/drive/MyDrive/ultralytics_datasets/Objects365/images/train/patch43

In [ ]:
!gzip -cd /content/drive/MyDrive/ultralytics_datasets/Objects365/zips/train/patch0.tar.gz | tar -tvv | grep -c ^-

In [ ]:
!ls /content/drive/MyDrive/ultralytics_datasets/Objects365/images/train/

In [ ]:
# === Objects365 extraction integrity check (follows notebook paths) ===
from pathlib import Path
import os, pandas as pd
from datetime import datetime

# Paths exactly as defined in your .ipynb
DRIVE_O365 = Path("/content/drive/MyDrive/ultralytics_datasets/Objects365")
ZIPS_DIR   = DRIVE_O365 / "zips"
TRAIN_ZIPS = ZIPS_DIR / "train"
VAL_ZIPS   = ZIPS_DIR / "val"
IM_TRAIN   = DRIVE_O365 / "images" / "train"
IM_VAL     = DRIVE_O365 / "images" / "val"

def get_size(path: Path) -> int:
    """Total size (bytes) for a file or a directory (recursive). Missing -> 0."""
    if not path.exists():
        return 0
    if path.is_file():
        return path.stat().st_size
    total = 0
    for root, _, files in os.walk(path):
        for f in files:
            fp = Path(root) / f
            try:
                total += fp.stat().st_size
            except FileNotFoundError:
                pass
    return total

def compare_split(split_name: str, tar_dir: Path, img_dir: Path):
    rows = []
    if not tar_dir.exists():
        print(f"[{split_name}] tar dir does not exist: {tar_dir}")
        return rows
    tar_files = sorted(tar_dir.glob("*.tar.gz"))
    print(f"[{split_name}] Found {len(tar_files)} tar.gz")
    for tar_path in tar_files:
        patch_name  = tar_path.stem.replace(".tar", "")  # e.g., 'patch123'
        patch_dir   = img_dir / patch_name               # assumes same top-level folder inside tar
        tar_bytes   = get_size(tar_path)
        fold_bytes  = get_size(patch_dir)
        ratio       = (fold_bytes / tar_bytes) if tar_bytes else 0.0
        status      = "OK" if (fold_bytes > tar_bytes) else "SMALLER_OR_MISSING"
        rows.append(dict(
            split=split_name,
            patch=patch_name,
            tar_path=str(tar_path),
            patch_folder=str(patch_dir),
            tar_size_bytes=tar_bytes,
            folder_size_bytes=fold_bytes,
            ratio_folder_to_tar=ratio,
            status=status
        ))
    return rows

all_rows = []
all_rows += compare_split("train", TRAIN_ZIPS, IM_TRAIN)
all_rows += compare_split("val",   VAL_ZIPS,   IM_VAL)

df = pd.DataFrame(all_rows)
if not df.empty:
    df = df.sort_values(["split","patch"]).reset_index(drop=True)

print("\n=== Summary ===")
if df.empty:
    print("No tarballs found under notebook paths. Check that Drive is mounted and zips are present.")
else:
    # Pretty print a compact table
    display(df[["split","patch","tar_size_bytes","folder_size_bytes","ratio_folder_to_tar","status"]].head(30))
    flagged = df[df["status"] == "SMALLER_OR_MISSING"]
    print(f"Flagged patches: {len(flagged)} / {len(df)}")
    if not flagged.empty:
        display(flagged[["split","patch","tar_path","patch_folder","tar_size_bytes","folder_size_bytes","ratio_folder_to_tar"]])

# Save CSV report next to the notebook (in /content by default) or in Drive; here we put it in Drive
reports_dir = DRIVE_O365 / "reports"
reports_dir.mkdir(parents=True, exist_ok=True)
out_csv = reports_dir / f"objects365_extraction_check_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
(df if not df.empty else pd.DataFrame(columns=[
    "split","patch","tar_path","patch_folder","tar_size_bytes","folder_size_bytes","ratio_folder_to_tar","status"
])).to_csv(out_csv, index=False)
print(f"\nSaved report to: {out_csv}")


In [ ]:
!pip -q install -U ultralytics pycocotools tqdm

from ultralytics import settings
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

# Put datasets and runs on Drive
DATASETS_DIR = "/content/drive/MyDrive/ultralytics_datasets"
RUNS_DIR     = "/content/drive/MyDrive/ultralytics_runs"
settings.update({"datasets_dir": DATASETS_DIR, "runs_dir": RUNS_DIR})
print("datasets_dir:", settings["datasets_dir"])
print("runs_dir    :", settings["runs_dir"])


In [ ]:
from pathlib import Path

# --- Root of your Objects365 on Drive (matches your screenshot) ---
DRIVE_O365 = Path("/content/drive/MyDrive/ultralytics_datasets/Objects365")

# Images and labels dirs
IM_TRAIN = DRIVE_O365 / "images/train"
IM_VAL   = DRIVE_O365 / "images/val"
LB_TRAIN = DRIVE_O365 / "labels/train"
LB_VAL   = DRIVE_O365 / "labels/val"

# Make sure label dirs exist (images should already exist after extraction)
LB_TRAIN.mkdir(parents=True, exist_ok=True)
LB_VAL.mkdir(parents=True, exist_ok=True)

print("Images train:", IM_TRAIN)
print("Images val  :", IM_VAL)
print("Labels train:", LB_TRAIN)
print("Labels val  :", LB_VAL)

# --- Annotations live at the ROOT (explicit paths) ---
TRAIN_JSON = DRIVE_O365 / "zhiyuan_objv2_train.json"
VAL_JSON   = DRIVE_O365 / "zhiyuan_objv2_val.json"

# Sanity checks
assert IM_TRAIN.exists(), f"Missing images/train: {IM_TRAIN}"
assert IM_VAL.exists(),   f"Missing images/val:   {IM_VAL}"
assert TRAIN_JSON.exists(), f"Missing train JSON at root: {TRAIN_JSON}\nIf you only have 'zhiyuan_objv2_train.tar.gz', extract it to this folder."
assert VAL_JSON.exists(),   f"Missing val JSON at root:   {VAL_JSON}"

print("TRAIN_JSON:", TRAIN_JSON)
print("VAL_JSON  :", VAL_JSON)


In [ ]:
import os, re, json
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pathlib import Path

# Your requested classes (exact names from YAML list)
TARGET_NAMES = [
    "Car","SUV","Bicycle","Van","Bus","Motorcycle","Truck","Pickup Truck",
    "Machinery Vehicle","Sports Car","Fire Truck","Heavy Truck","Ambulance"
]

MIN_BOX_AREA_PX = 4 * 4  # filter tiny boxes

def norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", s.strip().lower())

def load_coco(json_path: Path):
    with open(json_path, "r") as f:
        coco = json.load(f)
    cats = {c["id"]: c for c in coco["categories"]}
    imgs = {im["id"]: im for im in coco["images"]}
    anns_by_img = defaultdict(list)
    for a in coco["annotations"]:
        anns_by_img[a["image_id"]].append(a)
    # Build normalized name -> id map (handle minor case/spacing diffs)
    name2id_norm = {norm(c["name"]): cid for cid, c in cats.items()}
    return cats, imgs, anns_by_img, name2id_norm

def map_targets_to_ids(name2id_norm: dict, targets: list[str]):
    resolved, missing = {}, []
    for t in targets:
        key = norm(t)
        if key in name2id_norm:
            resolved[t] = name2id_norm[key]
        else:
            # fuzzy: substring both ways
            cand = None
            for k, cid in name2id_norm.items():
                if key in k or k in key:
                    cand = cid; break
            if cand is None:
                missing.append(t)
            else:
                resolved[t] = cand
    return resolved, missing

def normalize_rel_path(file_name: str, split: str):
    """Make path relative to images/<split>/ for label mirroring."""
    p = Path(file_name).as_posix().lstrip("/")
    # Strip common prefixes if present
    for prefix in [f"images/{split}/", f"{split}/", "images/"]:
        if p.startswith(prefix):
            p = p[len(prefix):]
    return p

def index_images(images_dir: Path):
    """Map: rel_path -> True, and basename -> [rel_paths] for fallback."""
    rel_exists = set()
    by_base = defaultdict(list)
    for root, _, files in os.walk(images_dir):
        rroot = Path(root).relative_to(images_dir)
        for fn in files:
            if fn.lower().endswith((".jpg",".jpeg",".png")):
                rel = (Path(rroot)/fn).as_posix()
                rel_exists.add(rel)
                by_base[fn].append(rel)
    return rel_exists, by_base

def write_label(im, anns, labels_root: Path, catid2idx: dict, split: str, rel_path: str):
    """Write one YOLO .txt label, mirroring the image's rel path."""
    out_path = labels_root / Path(rel_path).with_suffix(".txt")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    w, h = im["width"], im["height"]
    lines = []
    for a in anns:
        cid = a["category_id"]
        if cid not in catid2idx:
            continue
        x, y, bw, bh = a["bbox"]  # COCO xywh in pixels
        if bw <= 0 or bh <= 0 or (bw*bh) < MIN_BOX_AREA_PX:
            continue
        xc = (x + bw/2.0) / w
        yc = (y + bh/2.0) / h
        ww = bw / w
        hh = bh / h
        # clip
        xc = min(max(xc, 0.0), 1.0)
        yc = min(max(yc, 0.0), 1.0)
        ww = min(max(ww, 0.0), 1.0)
        hh = min(max(hh, 0.0), 1.0)
        lines.append(f"{catid2idx[cid]} {xc:.6f} {yc:.6f} {ww:.6f} {hh:.6f}")
    if lines:
        with open(out_path, "w") as f:
            f.write("\n".join(lines))
        return True
    return False

def convert_split(json_path: Path, images_dir: Path, labels_dir: Path, target_names: list[str], split: str):
    cats, imgs, anns_by_img, name2id_norm = load_coco(json_path)
    resolved, missing = map_targets_to_ids(name2id_norm, target_names)

    print(f"\n[{json_path.name}] target mapping:")
    for t in target_names:
        if t in resolved:
            print(f"  {t:18s} -> id {resolved[t]} ({cats[resolved[t]]['name']})")
        else:
            print(f"  {t:18s} -> NOT FOUND")
    if missing:
        print("⚠️ Missing targets (check spelling/case):", missing)

    keep_cids = set(resolved.values())
    catid2idx = {cid: i for i, cid in enumerate([resolved[t] for t in target_names if t in resolved])}
    yolo_names = [cats[cid]["name"] for cid in [resolved[t] for t in target_names if t in resolved]]

    # Build image indexes (existing rel paths + basename fallback)
    rel_exists, by_base = index_images(images_dir)

    # Select candidate image IDs that contain ≥1 of the target categories
    candidates = []
    for img_id, ann_list in anns_by_img.items():
        if any(a["category_id"] in keep_cids for a in ann_list):
            candidates.append(img_id)
    print(f"[{split}] images with target classes in annotations: {len(candidates):,}")

    # Multithread label creation with robust rel-path resolution
    used_rel_paths = []
    ok = 0
    def process(img_id):
        im = imgs[img_id]
        raw = im.get("file_name") or im.get("filename")
        rel = normalize_rel_path(raw, split)
        # If not found as-is, resolve by basename
        rel_resolved = rel
        if rel not in rel_exists:
            base = Path(rel).name
            cands = by_base.get(base, [])
            if len(cands) == 1:
                rel_resolved = cands[0]
            elif len(cands) > 1:
                # heuristic: pick the one whose parent contains a token from raw
                parent_tokens = set(Path(raw).parts)
                rel_resolved = next((c for c in cands if any(t in c for t in parent_tokens)), cands[0]) if cands else rel
            else:
                return False  # image file not on disk -> skip
        ann_keep = [a for a in anns_by_img[img_id] if a["category_id"] in keep_cids]
        wrote = write_label(im, ann_keep, labels_dir, catid2idx, split, rel_resolved)
        if wrote:
            used_rel_paths.append(rel_resolved)
        return wrote

    with ThreadPoolExecutor(max_workers=16) as ex:
        futures = [ex.submit(process, iid) for iid in candidates]
        for f in tqdm(as_completed(futures), total=len(futures), desc=f"Writing {split} labels"):
            ok += 1 if f.result() else 0

    print(f"Wrote {ok} label files to {labels_dir}")
    return yolo_names, used_rel_paths

# Run for train/val
names_train, used_train = convert_split(TRAIN_JSON, IM_TRAIN, LB_TRAIN, TARGET_NAMES, split="train")
names_val,   used_val   = convert_split(VAL_JSON,   IM_VAL,   LB_VAL,   TARGET_NAMES, split="val")

# Final names in YOUR requested order (from train mapping)
YOLO_NAMES = names_train
print("\nYOLO class order (index -> name):")
for i, n in enumerate(YOLO_NAMES):
    print(f" {i:2d} -> {n}")

# Save image lists (absolute paths) so Ultralytics reads nested images reliably
train_list = [str(IM_TRAIN / p) for p in used_train]
val_list   = [str(IM_VAL   / p) for p in used_val]
(Path(DRIVE_O365/"train_images.txt")).write_text("\n".join(train_list))
(Path(DRIVE_O365/"val_images.txt")).write_text("\n".join(val_list))
print(f"Saved lists:\n  {DRIVE_O365/'train_images.txt'}  ({len(train_list):,} lines)\n  {DRIVE_O365/'val_images.txt'}    ({len(val_list):,} lines)")

In [ ]:
import os, re, json
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pathlib import Path

# Your requested classes (exact names from YAML list)
TARGET_NAMES = [
    "Car","SUV","Bicycle","Van","Bus","Motorcycle","Truck","Pickup Truck",
    "Machinery Vehicle","Sports Car","Fire Truck","Heavy Truck","Ambulance"
]

MIN_BOX_AREA_PX = 4 * 4  # filter tiny boxes

def norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+","", s.strip().lower())

def load_coco(json_path: Path):
    with open(json_path, "r") as f:
        coco = json.load(f)
    cats = {c["id"]: c for c in coco["categories"]}
    imgs = {im["id"]: im for im in coco["images"]}
    anns_by_img = defaultdict(list)
    for a in coco["annotations"]:
        anns_by_img[a["image_id"]].append(a)
    # Build normalized name -> id map (handle minor case/spacing diffs)
    name2id_norm = {norm(c["name"]): cid for cid, c in cats.items()}
    return cats, imgs, anns_by_img, name2id_norm

def map_targets_to_ids(name2id_norm: dict, targets: list[str]):
    resolved, missing = {}, []
    for t in targets:
        key = norm(t)
        if key in name2id_norm:
            resolved[t] = name2id_norm[key]
        else:
            # fuzzy: substring both ways
            cand = None
            for k, cid in name2id_norm.items():
                if key in k or k in key:
                    cand = cid; break
            if cand is None:
                missing.append(t)
            else:
                resolved[t] = cand
    return resolved, missing

def normalize_rel_path(file_name: str, split: str):
    """Make path relative to images/<split>/ for label mirroring."""
    p = Path(file_name).as_posix().lstrip("/")
    # Strip common prefixes if present
    for prefix in [f"images/{split}/", f"{split}/", "images/"]:
        if p.startswith(prefix):
            p = p[len(prefix):]
    return p

def index_images(images_dir: Path):
    """Map: rel_path -> True, and basename -> [rel_paths] for fallback."""
    rel_exists = set()
    by_base = defaultdict(list)
    for root, _, files in os.walk(images_dir):
        rroot = Path(root).relative_to(images_dir)
        for fn in files:
            if fn.lower().endswith((".jpg",".jpeg",".png")):
                rel = (Path(rroot)/fn).as_posix()
                rel_exists.add(rel)
                by_base[fn].append(rel)
    return rel_exists, by_base

def write_label(im, anns, labels_root: Path, catid2idx: dict, split: str, rel_path: str):
    """Write one YOLO .txt label, mirroring the image's rel path."""
    out_path = labels_root / Path(rel_path).with_suffix(".txt")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    w, h = im["width"], im["height"]
    lines = []
    for a in anns:
        cid = a["category_id"]
        if cid not in catid2idx:
            continue
        x, y, bw, bh = a["bbox"]  # COCO xywh in pixels
        if bw <= 0 or bh <= 0 or (bw*bh) < MIN_BOX_AREA_PX:
            continue
        xc = (x + bw/2.0) / w
        yc = (y + bh/2.0) / h
        ww = bw / w
        hh = bh / h
        # clip
        xc = min(max(xc, 0.0), 1.0)
        yc = min(max(yc, 0.0), 1.0)
        ww = min(max(ww, 0.0), 1.0)
        hh = min(max(hh, 0.0), 1.0)
        lines.append(f"{catid2idx[cid]} {xc:.6f} {yc:.6f} {ww:.6f} {hh:.6f}")
    if lines:
        with open(out_path, "w") as f:
            f.write("\n".join(lines))
        return True
    return False

def convert_split(json_path: Path, images_dir: Path, labels_dir: Path, target_names: list[str], split: str):
    cats, imgs, anns_by_img, name2id_norm = load_coco(json_path)
    resolved, missing = map_targets_to_ids(name2id_norm, target_names)

    print(f"\n[{json_path.name}] target mapping:")
    for t in target_names:
        if t in resolved:
            print(f"  {t:18s} -> id {resolved[t]} ({cats[resolved[t]]['name']})")
        else:
            print(f"  {t:18s} -> NOT FOUND")
    if missing:
        print("⚠️ Missing targets (check spelling/case):", missing)

    keep_cids = set(resolved.values())
    catid2idx = {cid: i for i, cid in enumerate([resolved[t] for t in target_names if t in resolved])}
    yolo_names = [cats[cid]["name"] for cid in [resolved[t] for t in target_names if t in resolved]]

    # Build image indexes (existing rel paths + basename fallback)
    rel_exists, by_base = index_images(images_dir)

    # Select candidate image IDs that contain ≥1 of the target categories
    candidates = []
    for img_id, ann_list in anns_by_img.items():
        if any(a["category_id"] in keep_cids for a in ann_list):
            candidates.append(img_id)
    print(f"[{split}] images with target classes in annotations: {len(candidates):,}")

    # Multithread label creation with robust rel-path resolution
    used_rel_paths = []
    ok = 0
    def process(img_id):
        im = imgs[img_id]
        raw = im.get("file_name") or im.get("filename")
        rel = normalize_rel_path(raw, split)
        # If not found as-is, resolve by basename
        rel_resolved = rel
        if rel not in rel_exists:
            base = Path(rel).name
            cands = by_base.get(base, [])
            if len(cands) == 1:
                rel_resolved = cands[0]
            elif len(cands) > 1:
                # heuristic: pick the one whose parent contains a token from raw
                parent_tokens = set(Path(raw).parts)
                rel_resolved = next((c for c in cands if any(t in c for t in parent_tokens)), cands[0]) if cands else rel
            else:
                return False  # image file not on disk -> skip
        ann_keep = [a for a in anns_by_img[img_id] if a["category_id"] in keep_cids]
        wrote = write_label(im, ann_keep, labels_dir, catid2idx, split, rel_resolved)
        if wrote:
            used_rel_paths.append(rel_resolved)
        return wrote

    with ThreadPoolExecutor(max_workers=16) as ex:
        futures = [ex.submit(process, iid) for iid in candidates]
        for f in tqdm(as_completed(futures), total=len(futures), desc=f"Writing {split} labels"):
            ok += 1 if f.result() else 0

    print(f"Wrote {ok} label files to {labels_dir}")
    return yolo_names, used_rel_paths

# Run for train/val
names_train, used_train = convert_split(TRAIN_JSON, IM_TRAIN, LB_TRAIN, TARGET_NAMES, split="train")
names_val,   used_val   = convert_split(VAL_JSON,   IM_VAL,   LB_VAL,   TARGET_NAMES, split="val")

# Final names in YOUR requested order (from train mapping)
YOLO_NAMES = names_train
print("\nYOLO class order (index -> name):")
for i, n in enumerate(YOLO_NAMES):
    print(f" {i:2d} -> {n}")
# Save image lists (POSIX/forward-slash paths for robustness)
train_list = [(IM_TRAIN / p).as_posix() for p in used_train]
val_list   = [(IM_VAL   / p).as_posix() for p in used_val]

(DRIVE_O365 / "train_images.txt").write_text("\n".join(train_list), encoding="utf-8")
(DRIVE_O365 / "val_images.txt").write_text("\n".join(val_list),   encoding="utf-8")

print(f"Saved lists:\n  {DRIVE_O365/'train_images.txt'}  ({len(train_list):,} lines)\n  {DRIVE_O365/'val_images.txt'}    ({len(val_list):,} lines)")


In [ ]:
import os, shutil, random
from pathlib import Path
from collections import Counter

# ==== CONFIG ====
DATA_ROOT   = Path(r"/content/drive/MyDrive/ultralytics_datasets/Objects365/")
IM_TRAIN    = DATA_ROOT / "images" / "train"
IM_VAL      = DATA_ROOT / "images" / "val"
LB_TRAIN    = DATA_ROOT / "labels" / "train"
LB_VAL      = DATA_ROOT / "labels" / "val"

# Subset destination
SUBSET_ROOT = Path(r"/content/drive/MyDrive/ultralytics_datasets/O365_vehicle13_subset")
S_IM_TRN    = SUBSET_ROOT / "images" / "train"
S_IM_VAL    = SUBSET_ROOT / "images" / "val"
S_LB_TRN    = SUBSET_ROOT / "labels" / "train"
S_LB_VAL    = SUBSET_ROOT / "labels" / "val"
for p in [S_IM_TRN, S_IM_VAL, S_LB_TRN, S_LB_VAL]:
    p.mkdir(parents=True, exist_ok=True)

# YOLO class list in the order you created labels (0..12)
YOLO_NAMES = [
    "Car","SUV","Bicycle","Van","Bus","Motorcycle","Truck","Pickup Truck",
    "Machinery Vehicle","Sports Car","Fire Truck","Heavy Truck","Ambulance"
]
TARGET_CLASS_IDS = set(range(len(YOLO_NAMES)))  # {0..12}

# Caps — tune these to shrink the set
CAP_PER_CLASS_TRAIN = 10000   # e.g., keep up to 10k images per class for train
CAP_PER_CLASS_VAL   = 2000    # e.g., keep up to 2k images per class for val
MAX_TOTAL_TRAIN     = None    # or an integer to hard-cap total train images
MAX_TOTAL_VAL       = None    # or an integer to hard-cap total val images

USE_HARDLINKS = True  # hard link (fast, no extra disk) on same drive; falls back to copy on failure

# ==== UTIL ====
def label_path_from_img(img_path: Path, split: str) -> Path:
    # convert ...\images\{split}\...jpg to ...\labels\{split}\...txt
    rel = img_path.relative_to(IM_TRAIN if split=="train" else IM_VAL)
    return (LB_TRAIN if split=="train" else LB_VAL) / rel.with_suffix(".txt")

def read_classes_in_label(lbl_path: Path):
    """Return a set of class ids present in a YOLO .txt label file."""
    s = set()
    if not lbl_path.exists():
        return s
    with open(lbl_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            parts=line.split()
            try:
                cid = int(parts[0])
            except Exception:
                continue
            s.add(cid)
    return s

def link_or_copy(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists():
        return
    try:
        if USE_HARDLINKS:
            os.link(src, dst)  # hard link (same volume)
        else:
            shutil.copy2(src, dst)
    except Exception:
        shutil.copy2(src, dst)

def choose_subset(split: str, cap_per_class: int, max_total=None, shuffle=True):
    """
    Build a balanced subset:
    - parse labels to know which classes are present per image
    - allocate each image to the 'rarest' class it contains (to balance)
    - stop at per-class cap
    """
    if split == "train":
        img_root, lb_root, s_im_root, s_lb_root = IM_TRAIN, LB_TRAIN, S_IM_TRN, S_LB_TRN
    else:
        img_root, lb_root, s_im_root, s_lb_root = IM_VAL, LB_VAL, S_IM_VAL, S_LB_VAL

    # collect all label files under labels/{split}
    label_files = list(lb_root.rglob("*.txt"))
    if shuffle:
        random.Random(1337).shuffle(label_files)

    class_counts = Counter()
    selected = []  # list of (img_path, lbl_path)
    for lbl in label_files:
        # infer image path by mirroring labels->images
        rel = lbl.relative_to(lb_root)
        img = (img_root / rel).with_suffix(".jpg")
        if not img.exists():
            # try common alternate extensions
            for ext in [".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]:
                if (img_root / rel).with_suffix(ext).exists():
                    img = (img_root / rel).with_suffix(ext)
                    break
            else:
                continue
        # read present classes
        present = read_classes_in_label(lbl) & TARGET_CLASS_IDS
        if not present:
            continue

        # choose the rarest present class to balance
        rarest = min(present, key=lambda c: class_counts[c])
        if class_counts[rarest] >= cap_per_class:
            continue

        selected.append((img, lbl))
        class_counts[rarest] += 1

        if max_total and len(selected) >= max_total:
            break

        # early exit if all classes reached cap
        if all(class_counts[c] >= cap_per_class for c in TARGET_CLASS_IDS):
            break

    # materialize (hard-link/copy) into subset
    for img, lbl in selected:
        rel_img = img.relative_to(img_root)
        dst_img = s_im_root / rel_img
        link_or_copy(img, dst_img)

        rel_lbl = lbl.relative_to(lb_root)
        dst_lbl = s_lb_root / rel_lbl
        link_or_copy(lbl, dst_lbl)

    print(f"[{split}] -> kept {len(selected)} images")
    print(" per-class counts:", dict(class_counts))
    return selected

sel_train = choose_subset("train", cap_per_class=CAP_PER_CLASS_TRAIN, max_total=MAX_TOTAL_TRAIN)
sel_val   = choose_subset("val",   cap_per_class=CAP_PER_CLASS_VAL,   max_total=MAX_TOTAL_VAL)

# Write subset image lists (forward slashes)
train_list = [(S_IM_TRN / (p[0].relative_to(IM_TRAIN))).as_posix() for p in sel_train]
val_list   = [(S_IM_VAL / (p[0].relative_to(IM_VAL))).as_posix()   for p in sel_val]

(SUBSET_ROOT / "train_images.txt").write_text("\n".join(train_list), encoding="utf-8")
(SUBSET_ROOT / "val_images.txt").write_text("\n".join(val_list), encoding="utf-8")

print("Subset lists:")
print(" ", SUBSET_ROOT / "train_images.txt", len(train_list))
print(" ", SUBSET_ROOT / "val_images.txt",   len(val_list))


In [ ]:
import yaml, warnings
from PIL import Image
# Optional: disable PIL DecompressionBomb warnings (large panoramas etc.)
Image.MAX_IMAGE_PIXELS = None
warnings.filterwarnings("ignore", category=UserWarning, module="PIL.Image")

DATASET_YAML = SUBSET_ROOT / "o365_vehicle13_subset.yaml"
yaml.safe_dump({
    "train": (SUBSET_ROOT / "train_images.txt").as_posix(),
    "val":   (SUBSET_ROOT / "val_images.txt").as_posix(),
    "nc": len(YOLO_NAMES),
    "names": YOLO_NAMES
}, open(DATASET_YAML, "w", encoding="utf-8"), sort_keys=False)

print("Wrote:", DATASET_YAML)
print(DATASET_YAML.read_text()[:400])


In [ ]:
! ls "/content/drive/MyDrive/ultralytics_datasets/Objects365/"

In [ ]:
!unzip -o "/content/drive/MyDrive/ultralytics_datasets/Objects365/labels0.zip" -d "/content/drive/MyDrive/ultralytics_datasets/Objects365/labels/"

In [ ]:
!ls -lh "/content/drive/MyDrive/ultralytics_datasets/Objects365/labels0.zip"

In [ ]:
!mv /content/drive/MyDrive/ultralytics_datasets/Objects365/labels/labels/* /content/drive/MyDrive/ultralytics_datasets/Objects365/labels/


In [ ]:
!ls -lh "/content/drive/MyDrive/ultralytics_datasets/Objects365/labels/train"

In [ ]:
!find "/content/drive/MyDrive/ultralytics_datasets/Objects365/labels/val"   -type f -name "*.txt" | wc -l
!find "/content/drive/MyDrive/ultralytics_datasets/Objects365/labels/train" -type f -name "*.txt" | wc -l

In [ ]:
import yaml

DATASET_YAML = DRIVE_O365/"objects365_vehicle13_on_drive.yaml"
yaml_obj = {
    "train": str(DRIVE_O365/"train_images.txt"),  # text files of absolute image paths
    "val":   str(DRIVE_O365/"val_images.txt"),
    "nc": len(YOLO_NAMES),
    "names": YOLO_NAMES
}
with open(DATASET_YAML, "w") as f:
    yaml.safe_dump(yaml_obj, f, sort_keys=False)

print("Wrote dataset YAML:", DATASET_YAML)
print(DATASET_YAML.read_text()[:400])


In [ ]:
!head -n 10 /content/drive/MyDrive/ultralytics_datasets/Objects365/train_images.txt

In [ ]:
import os
os.cpu_count()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [23]:
!pip install -U ultralytics --break-system-packages

In [12]:
from ultralytics import YOLO
import pandas as pd
from pathlib import Path

SUBSET_ROOT = Path("/workspace/O365_vehicle13_subset_2")
DATASET_YAML = SUBSET_ROOT / "o365_vehicle13_subset.yaml"
DATA_YAML = str(DATASET_YAML)
PROJECT   = "/content/ultralytics_runs"
IMG_SIZE  = 640      # go 1536 or 1920 if you can afford it
EPOCHS    = 300
WORKERS   = 64

models = [
    # ("/content/ultralytics_runs/yolov8x/weights/last.pt",  "yolov8x"),
    # ("/content/ultralytics_runs/yolo11x/weights/last.pt",  "yolo11x"),
    ("yolo12x.pt",  "yolo12x"),  # skipped gracefully if not available in your wheel
]

common = dict(
    data=DATA_YAML,
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=-1,          # auto-batch for A100
    device=0,
    optimizer="AdamW",
    cos_lr=True,
    lr0=0.001,
    weight_decay=0.0005,
    patience=30,
    amp=False,
    workers=WORKERS,
    close_mosaic=10,
    cache=True,       # set True only if subset fits RAM/SSD
    project=PROJECT,
    exist_ok=True,
    save_period = 1,
    # resume=True,
)

rows, best_ckpts = [], {}
for w, tag in models:
    print(f"\n=== Training {tag} ===")
    try:
        model = YOLO(w)
        model.train(name=tag, **common)
        m = model.val(data=DATA_YAML, imgsz=IMG_SIZE)
        rows.append({"model": tag,
                     "mAP50-95": float(m.box.map),
                     "mAP50": float(m.box.map50),
                     "mAP75": float(m.box.map75)})
        best_ckpts[tag] = str(Path(model.ckpt_path))
        print(f"[OK] {tag} best: {best_ckpts[tag]}")
    except Exception as e:
        print(f"[SKIP] {tag}: {e}")

df = pd.DataFrame(rows).sort_values("mAP50-95", ascending=False)
cmp_csv = Path(RUNS_DIR)/PROJECT/"model_comparison.csv"
cmp_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(cmp_csv, index=False)
display(df)
print("Saved comparison CSV to:", cmp_csv)



=== Training yolo12x ===
Ultralytics 8.3.228 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA H200 NVL, 143156MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/workspace/O365_vehicle13_subset_2/o365_vehicle13_subset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo12x, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, o

NameError: name 'RUNS_DIR' is not defined

In [ ]:
from ultralytics import YOLO
import glob

if best_ckpts:
    best = max(rows, key=lambda r: r["mAP50-95"])["model"]
    print("Best model:", best, "->", best_ckpts[best])
    model = YOLO(best_ckpts[best])

    # Pick a few validation images
    sample_imgs = []
    for ext in ("*.jpg","*.jpeg","*.png"):
        sample_imgs += glob.glob(str(IM_VAL / "**" / ext), recursive=True)
    sample_imgs = sample_imgs[:8]
    print("Predicting on", len(sample_imgs), "images...")

    model.predict(
        source=sample_imgs,
        imgsz=IMG_SIZE,
        conf=0.35,
        save=True,
        project=PROJECT,
        name="predict_samples",
        device=0
    )
    print("Predictions saved under:", Path(RUNS_DIR)/PROJECT/"predict_samples")


In [13]:
from ultralytics import YOLO
import pandas as pd
from pathlib import Path
import glob

SUBSET_ROOT = Path("/workspace/O365_vehicle13_subset_2")
DATASET_YAML = SUBSET_ROOT / "o365_vehicle13_subset.yaml"
DATA_YAML = str(DATASET_YAML)
PROJECT   = "/content/ultralytics_runs"
IMG_SIZE  = 640      # go 1536 or 1920 if you can afford it
EPOCHS    = 300
WORKERS   = 64

# Define this if not already:
IM_VAL = SUBSET_ROOT / "images" / "val"   # adjust if your val path differs

models = [
    ("/content/ultralytics_runs/yolov8x/weights/last.pt",  "yolov8x"),
    ("/content/ultralytics_runs/yolo11x/weights/last.pt",  "yolo11x"),
    ("/content/ultralytics_runs/yolo12x/weights/last.pt",  "yolo12x"),  # skipped gracefully if not available in your wheel
]

common = dict(
    data=DATA_YAML,
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=-1,          # auto-batch
    device=0,
    optimizer="AdamW",
    cos_lr=True,
    lr0=0.003,
    weight_decay=0.0005,
    patience=30,
    workers=WORKERS,
    cache=True,       # True only if subset fits RAM/SSD
    project=PROJECT,
    exist_ok=True,    # ok to keep
    resume=True,      # resume interrupted runs
)

rows, best_ckpts = [], {}

for w, tag in models:
    print(f"\n=== Training {tag} ===")
    try:
        model = YOLO(w)
        model.train(name=tag, **common)

        # --- NEW: explicitly use best.pt for validation ---
        run_dir   = Path(PROJECT) / tag
        best_path = run_dir / "weights" / "best.pt"

        if best_path.is_file():
            print(f"Using best checkpoint for {tag}: {best_path}")
            eval_model = YOLO(best_path)
            best_ckpts[tag] = str(best_path)
        else:
            # Fallback to the current model (last.pt) if best.pt not found
            print(f"[WARN] best.pt not found for {tag}, using current model weights")
            eval_model = model
            best_ckpts[tag] = str(Path(w))

        m = eval_model.val(data=DATA_YAML, imgsz=IMG_SIZE)

        rows.append({
            "model": tag,
            "mAP50-95": float(m.box.map),
            "mAP50": float(m.box.map50),
            "mAP75": float(m.box.map75),
        })

        print(f"[OK] {tag} best ckpt: {best_ckpts[tag]}")

    except Exception as e:
        print(f"[SKIP] {tag}: {e}")

# --- Save comparison CSV ---
df = pd.DataFrame(rows).sort_values("mAP50-95", ascending=False)
cmp_csv = Path(PROJECT) / "model_comparison.csv"
cmp_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(cmp_csv, index=False)
display(df)
print("Saved comparison CSV to:", cmp_csv)

# --- Use the globally best model for sample predictions ---
if best_ckpts:
    best_row  = df.iloc[0]
    best_name = best_row["model"]
    best_path = best_ckpts[best_name]

    print("Best model:", best_name, "->", best_path)
    model = YOLO(best_path)

    # Pick a few validation images
    sample_imgs = []
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        sample_imgs += glob.glob(str(IM_VAL / "**" / ext), recursive=True)
    sample_imgs = sample_imgs[:8]
    print("Predicting on", len(sample_imgs), "images...")

    model.predict(
        source=sample_imgs,
        imgsz=IMG_SIZE,
        conf=0.35,
        save=True,
        project=PROJECT,
        name="predict_samples",
        device=0,
    )

    print("Predictions saved under:", Path(PROJECT) / "predict_samples")



=== Training yolov8x ===
Ultralytics 8.3.228 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA H200 NVL, 143156MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/workspace/O365_vehicle13_subset_2/o365_vehicle13_subset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/ultralytics_runs/yolov8x/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8x, nbs=64, nms=False, opset=No

Exception in thread Thread-298 (_pin_memory_loop):
Traceback (most recent call last):
  File "/venv/main/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/venv/main/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/venv/main/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 52, in _pin_memory_loop
    do_one_step()
  File "/venv/main/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/main/lib/python3.12/site-packages/torch/multiproces

Ultralytics 8.3.228 🚀 Python-3.12.11 torch-2.9.1+cu128 CUDA:0 (NVIDIA H200 NVL, 143156MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/workspace/O365_vehicle13_subset_2/o365_vehicle13_subset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/ultralytics_runs/yolo12x/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo12x, nbs=64, nms=False, opset=None, optimize=False, optim

KeyError: 'mAP50-95'